In [55]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
from scipy.spatial.distance import cosine

Today we will focus on finding similarities between documents. For this purpose, we will compare the content of these documents. The same techniques can be used for a query in a search engine. Then simply we can treat the query like another document, calculate similarities and return the most similar documents.

In [56]:
documents = ['Machine Learning',
 'Five Advanced Plots in Python - Matplotlib',
 'How to Make your Computer Talk with Python',
 'Anomaly Detection on Servo Drives',
 'Key takeaways from Kaggle’s most recent time series competition - Ventilator Pressure Prediction',
 'Animated Mathematical Analysis',
 'How to Perform Speech Recognition with Python',
 'Beyond The Semesters: E04',
 'How to improve classification of e-commerce pages, incorporating multiple modalities',
 'Time Series Forecasting with ThymeBoost',
 'CHAPTER 2: Why I Chose Data Science!',
 'Training Provably-Robust Neural Networks',
 'Time Series Forecasting with ThymeBoost',
 'How to improve classification of e-commerce pages, incorporating multiple modalities',
 '5 Cute Features of CatBoost',
 'Variance Inflation Factor (VIF) and it’s relationship with multicollinearity&nbsp;.',
 'Beyond The Semesters: E04',
 'Efficient Digital Transformation - Particle Swarm Optimiser',
 'MEASURE OF ASYMMETRY',
 'What is linear regression? A quick cover with a tutorial',
 'Correlation VS Covariance: The easy way',
 'Are Recommender System harming us?',
 '1 Line of Python Code That Will Speed Up Your AI by Up to 6x',
 'If You Are Serious About Data Science Job. You Must Know These 3 Things.',
 'Recommender System With Machine Learning and Statistics',
 'Bias detection and mitigation in IBM AutoAI',
 'Data Engineering: Create your own Dataset',
 'Graph Neural Networks and Generalizable Models in Neuroscience',
 'Fastest Way of Deploying Your Machine Learning Models',
 'A Novel Approach to Integrate Speech Recognition into Authentication Systems',
 '3 Lessons Learned in Teaching Machine Learning for Earth Observation Techniques',
 'Vision Transformer in Galaxy Morphology Classification',
 'Exploring Methods of Deep Reinforcement Learning with NLP Applications',
 '6 Essential Tips to Solve Data Science Projects',
 'Data Science Interview Questions My Friends and I got asked recently (III)',
 'Understanding Uber’s Generative Teaching Networks',
 'How to achieve efficient large-batch training?',
 'How Parallelization and Large Batch Size Improve the Performance of Deep Neural Networks.',
 'Why You Need to Know the Inner Workings of Models',
 'Let’s Build A Simple Object Classification Task I']

In [57]:
CountVec = CountVectorizer(ngram_range=(1,1), stop_words='english')
CountData = CountVec.fit_transform(documents)

CountData

<40x150 sparse matrix of type '<class 'numpy.int64'>'
	with 204 stored elements in Compressed Sparse Row format>

The very basic way of storing information about documents is word count. Simply for each document we store an information how many times each word appears. It can be stored in an array, however, it's not the best option since it will be filled mostly with 0s. That's why it's stored in a sparse matrix, but we can expand it.

In [58]:
df=pd.DataFrame(CountData.toarray(), columns=CountVec.get_feature_names_out(), index=documents)
df

,6x,achieve,advanced,ai,analysis,animated,anomaly,applications,approach,asked,...,tutorial,uber,understanding,variance,ventilator,vif,vision,vs,way,workings
Machine Learning,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Five Advanced Plots in Python - Matplotlib,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
How to Make your Computer Talk with Python,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Anomaly Detection on Servo Drives,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Key takeaways from Kaggle’s most recent time series competition - Ventilator Pressure Prediction,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Animated Mathematical Analysis,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
How to Perform Speech Recognition with Python,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Beyond The Semesters: E04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"How to improve classification of e-commerce pages, incorporating multiple modalities",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Time Series Forecasting with ThymeBoost,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Task 1
We can reduce the size of an array, get rid of unnecesary words, and improve the quality of comparison by firstly preprocessing the docuemnts.
Check array size after stemming/lemmatization and without stop words

In [59]:
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.corpus import stopwords

def stem_documents(documents):
    porter = PorterStemmer()
    documents_stemmed = []
    for doc in documents:
        stemmed = porter.stem(doc)
        stop = stopwords.words('english')
        stems = {stemmed}.difference(set(stop))
        documents_stemmed.append(list(stems)[0])
    return documents_stemmed

documents_stemmed = stem_documents(documents)
CountVec = CountVectorizer(ngram_range=(1,1), stop_words='english')
CountData = CountVec.fit_transform(documents_stemmed)

df=pd.DataFrame(CountData.toarray(), columns=CountVec.get_feature_names_out(), index=documents)

In [60]:
df

,6x,achieve,advanced,ai,analysi,animated,anomaly,appl,approach,asked,...,tutori,uber,understanding,variance,ventilator,vif,vision,vs,way,workings
Machine Learning,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Five Advanced Plots in Python - Matplotlib,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
How to Make your Computer Talk with Python,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Anomaly Detection on Servo Drives,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Key takeaways from Kaggle’s most recent time series competition - Ventilator Pressure Prediction,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Animated Mathematical Analysis,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
How to Perform Speech Recognition with Python,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Beyond The Semesters: E04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"How to improve classification of e-commerce pages, incorporating multiple modalities",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Time Series Forecasting with ThymeBoost,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Task 2

Easy technique to compare two documents is a jaccard similarity.
$J={\frac {|A\cap B|}{|A\cup B|}}.$

Implement Jaccard similarity, and function finding closest document to a provided query. Test different queries

In [64]:
def jaccard(d1, d2):
    union = set(d1.split()).union(set(d2.split()))
    intersect = set(d1.split()).intersection(set(d2.split()))
    return len(intersect)/len(union)

def closest(query, df):
    max_sim = 0
    max_doc = None
    for doc in df:
        sim = jaccard(query, doc)
        if max_sim < sim:
            max_sim = sim
            max_doc = doc
    return max_doc

<a href="https://ibb.co/k4rRpf9"><img src="https://i.ibb.co/GW1KXLt/ir4.jpg" alt="ir4" border="0"></a>

In [65]:
queries = [
    "python",
    "plot neural network",
    "plot neural networks",
    "ploting neural networks",
    "data science",
]
queries = stem_documents(queries)
for q in queries:
    print(q)
    print(closest(q, documents_stemmed))

python
five advanced plots in python - matplotlib
plot neural network
training provably-robust neural network
plot neural network
training provably-robust neural network
ploting neural network
training provably-robust neural network
data sci
data engineering: create your own dataset


## Task 3

TFIDF (term frequency–inverse document frequency) is a much better approach. The tf–idf value increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word, which helps to adjust for the fact that some words appear more frequently in general.

This approach consists of 2 steps:
TF (term frequency) -  $tf(t,d)$, is the relative frequency of term $t$ within document $d$, can be expressed e.g. as a word count divided by number of terms in a given document or by the maximum term count in a given document.

IDF (inverse document frequency) - is a measure of how much information the word provides. If a word appears in every document it does not provide much information, but if it just appears in two documents then its impact on similiarity between these two documents is higher. The standard approach to compute this value is logarithm of number of documents divided by number of documents containing a given term $IDF(t) = log(\frac{N}{n_t})$

TFIDF is then just TF multiplied by IDF


Implement tf idf, compare it with sklearn TfidfVectorizer

In [45]:
tfidf=TfidfVectorizer(use_idf=True, smooth_idf=False)

dfTFIDF = pd.DataFrame(tfidf.fit_transform(documents).toarray(), index=documents, columns=tfidf.get_feature_names_out())
dfTFIDF

,6x,about,achieve,advanced,ai,analysis,and,animated,anomaly,applications,...,vision,vs,way,what,why,will,with,workings,you,your
Machine Learning,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Five Advanced Plots in Python - Matplotlib,0.000000,0.000000,0.000000,0.450495,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
How to Make your Computer Talk with Python,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.249731,0.000000,0.000000,0.316067
Anomaly Detection on Servo Drives,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.459985,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Key takeaways from Kaggle’s most recent time series competition - Ventilator Pressure Prediction,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Animated Mathematical Analysis,0.000000,0.000000,0.000000,0.000000,0.000000,0.57735,0.000000,0.57735,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
How to Perform Speech Recognition with Python,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.280999,0.000000,0.000000,0.000000
Beyond The Semesters: E04,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"How to improve classification of e-commerce pages, incorporating multiple modalities",0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Time Series Forecasting with ThymeBoost,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.324860,0.000000,0.000000,0.000000


In [46]:
pd.Series(tfidf.idf_, index=tfidf.get_feature_names_out()).sort_values()

of           2.491655
to           2.491655
with         2.609438
and          2.897120
how          2.897120
               ...   
integrate    4.688879
interview    4.688879
into         4.688879
need         4.688879
measure      4.688879
Length: 184, dtype: float64

In [47]:
query = "how to machine learning"
query = tfidf.transform([query]).toarray()[0]
1-dfTFIDF.apply(lambda x: cosine(x, query), axis=1).sort_values()

Machine Learning                                                                                    0.763355
Recommender System With Machine Learning and Statistics                                             0.364334
Fastest Way of Deploying Your Machine Learning Models                                               0.328159
How to Perform Speech Recognition with Python                                                       0.265814
3 Lessons Learned in Teaching Machine Learning for Earth Observation Techniques                     0.258540
How to achieve efficient large-batch training?                                                      0.246288
How to Make your Computer Talk with Python                                                          0.236235
How to improve classification of e-commerce pages, incorporating multiple modalities                0.221282
How to improve classification of e-commerce pages, incorporating multiple modalities                0.221282
Exploring Methods o

## Task 4
Create a search engine based on TFIDF

In [66]:
def search(query, df):
    tfidf=TfidfVectorizer(use_idf=True, smooth_idf=False)
    dfTFIDF = pd.DataFrame(tfidf.fit_transform(df.index).toarray(), index=df.index, columns=tfidf.get_feature_names_out())
    query = tfidf.transform([query]).toarray()[0]
    return 1-dfTFIDF.apply(lambda x: cosine(x, query), axis=1).sort_values()

In [68]:
search("Machine Learning", df)

Machine Learning                                                                                    1.000000
Recommender System With Machine Learning and Statistics                                             0.477280
Fastest Way of Deploying Your Machine Learning Models                                               0.429890
3 Lessons Learned in Teaching Machine Learning for Earth Observation Techniques                     0.338689
Exploring Methods of Deep Reinforcement Learning with NLP Applications                              0.172396
1 Line of Python Code That Will Speed Up Your AI by Up to 6x                                        0.000000
If You Are Serious About Data Science Job. You Must Know These 3 Things.                            0.000000
Bias detection and mitigation in IBM AutoAI                                                         0.000000
Data Engineering: Create your own Dataset                                                           0.000000
Graph Neural Networ

## Task 5
Create a search engine based on history containing more than one document

In [ ]:
def search_engine(history, df):
    pass